<div style="color: green; font-weight:
bold">The solution does not produce any output (without errors). Because of that we cannot comment on the actual outcome. <br>
The kernel unction is not implemented correctly as the square root of the pairwise_distance was not calculated. <br>
The matrix A was computed on the distance of all trainnig pixels times all training pixels, however, this yields a bad performance. It should have been calculated the diagonals and then add the transpose of the matrix to itself, as in the sample solution. Functionality wise this shouldn't affect the solution. <br>
For calculating y_hat, the nadaraya watson regression implemented as well as gpr. However, the vector matrix b was calculated wrongly as the kernel function was used only on the test points and not the test points times the training points. 
Then the gpr and the nadaraya-watson regression were implemeted correctly. The errors before made the calculation impossible though, leading to the error faced when executing the code
<br>
    
</div>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
from scipy.linalg import solve_banded


def load_image(filename):
    image = plt.imread(filename)
    return image


def preprocess_image(image):
    # Flatten the image and normalize gray values to the range [0, 1]
    flattened_image = image.flatten() / 255.0
    return flattened_image


def construct_kernel_matrix(X, gamma, h):
    # Reshape the coordinates to a 2D array
    X = X.reshape(-1, 2)

    # Compute the squared Euclidean distances between points
    pairwise_distances = np.sum((X[:, None] - X) ** 2, axis=2)

    # Compute the radial basis function kernel matrix
    kernel_matrix = np.exp(-pairwise_distances / (h ** 2)) ** gamma
    return kernel_matrix


def reconstruct_image(image, kernel_gamma, kernel_h, noise_variance):
    # Preprocess the image
    flattened_image = preprocess_image(image)

    # Get the coordinates of the known and missing pixels
    coordinates = np.argwhere(flattened_image > 0)
    known_pixels = flattened_image[flattened_image > 0]
    missing_pixels = flattened_image[flattened_image == 0]

    # Construct the kernel matrix A
    kernel_matrix = construct_kernel_matrix(coordinates, kernel_gamma, kernel_h)
    n = kernel_matrix.shape[0]

    # Add noise variance to the diagonal of the kernel matrix
    kernel_matrix[np.diag_indices(n)] += noise_variance

    # Construct the similarity vector b for the missing pixels
    similarity_vector = construct_kernel_matrix(missing_pixels, kernel_gamma, kernel_h) @ coordinates.T
    similarity_vector /= similarity_vector.sum(axis=1, keepdims=True)

    # Solve the linear system (A + σ^2 * I) · z = y
    banded_matrix = diags([kernel_matrix.diagonal()], [0], shape=(n, n))
    z = solve_banded((1, 1), banded_matrix.toarray(), known_pixels)

    # Reconstruct the missing pixel values
    reconstructed_pixels = similarity_vector @ z

    # Replace the missing pixels in the original image
    reconstructed_image = flattened_image.copy()
    reconstructed_image[reconstructed_image == 0] = reconstructed_pixels

    # Reshape the image back to its original shape
    reconstructed_image = reconstructed_image.reshape(image.shape)
    return reconstructed_image








def nadaraya_watson_regression(image):
    # Preprocess the image
    flattened_image = preprocess_image(image)

    # Get the coordinates of the known and missing pixels
    coordinates = np.argwhere(flattened_image > 0)
    known_pixels = flattened_image[flattened_image > 0]
    missing_pixels = flattened_image[flattened_image == 0]

    # Construct the similarity vector b for the missing pixels
    similarity_vector = construct_kernel_matrix(coordinates, 1, 1) @ coordinates.T
    similarity_vector /= similarity_vector.sum(axis=0)
    similarity_vector = similarity_vector.T

    # Reconstruct the missing pixel values using Nadaraya-Watson regression
    reconstructed_pixels = similarity_vector @ known_pixels / np.linalg.norm(similarity_vector, ord=1)

    # Replace the missing pixels in the original image
    reconstructed_image = flattened_image.copy()
    reconstructed_image[reconstructed_image == 0] = reconstructed_pixels

    # Reshape the image back to its original shape
    reconstructed_image = reconstructed_image.reshape(image.shape)
    return reconstructed_image


# Load the image
image = load_image("../cc_90.png")

# Reconstruct the image using Gaussian process regression
reconstructed_image = reconstruct_image(image, 1, 1, 0.01)

# Reconstruct the image using Nadaraya-Watson regression
nw_reconstructed_image = nadaraya_watson_regression(image)

# Display the original image and the reconstructed images
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.title("Original Image")

plt.subplot(1, 3, 2)
plt.imshow(reconstructed_image, cmap='gray')
plt.title("Gaussian Process Regression")

plt.subplot(1, 3, 3)
plt.imshow(nw_reconstructed_image, cmap='gray')
plt.title("Nadaraya-Watson Regression")

plt.show()


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 38025)